<a href="https://colab.research.google.com/github/Munya07/NLP/blob/master/3_Chat_bot/Deep_pavlov/Chatbot_deeppavlov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q deeppavlov

In [0]:
!pip install geopy
!pip install python-forecastio
!pip install darkskylib
!pip install dateparser

In [0]:
from deeppavlov.contrib.skills.similarity_matching_skill import SimilarityMatchingSkill
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from geopy.geocoders import Nominatim
from darksky import forecast
from datetime import date, timedelta
import dateparser


In [0]:
from deeppavlov.skills.pattern_matching_skill import PatternMatchingSkill
hello = PatternMatchingSkill(responses=['Hi, how can l help you'], patterns=['hi', 'hello', 'How are you'], default_confidence=0.3)
bye = PatternMatchingSkill(responses=['Good bye'], patterns=['bye bye', 'bye'], default_confidence=0.3)
weather = PatternMatchingSkill(responses=['For which location and date, l can only forecast for 1 week'], patterns=['weather', 'rain','temperature'], default_confidence=0.3)
check=PatternMatchingSkill(responses=['Let me check'], patterns=['','what', 'today', 'tomorrow'], default_confidence=0.2)

In [0]:
from deeppavlov.agents.default_agent.default_agent import DefaultAgent
from deeppavlov.agents.processors.highest_confidence_selector import HighestConfidenceSelector
agent = DefaultAgent([hello, bye, weather,check], skills_selector=HighestConfidenceSelector())

In [0]:

  
def run(input): 
  doc = nlp(input)
  dct = dict(( X.label_,X.text) for X in doc.ents)
  location = dct.get('GPE')
  date=dct.get('DATE')
  
    
    
  if location!=None and date!=None:

    
    import dateparser
    d=dateparser.parse(date)
    d=str(d)
    d=d[:10]
    d=datetime.datetime.strptime(d,'%Y-%m-%d').strftime('%m/%d/%y')
    
    from darksky import forecast
    from datetime import date, timedelta

    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="specify_your_app_name_here")
    location_ = geolocator.geocode(location)

    BOSTON = location_.latitude, location_.longitude

    weekday = date.today()
    with forecast('d6c7239325b1a5e9543d57c7cd385ff4', *BOSTON) as boston:
      print('In Summary ',boston.daily.summary, end='\n---\n')
      for day in boston.daily:
          day = dict(day = date.strftime(weekday, '%x'),
                     sum = day.summary,
                     tempMin = day.temperatureMin,
                     tempMax = day.temperatureMax)
          if day.get('day')== d:
            print('{day}: {sum} Temp range: {tempMin} - {tempMax}'.format(**day))
          weekday += timedelta(days=1)
      
    
  else:
    if location==None and date!=None:
      print('sorry your location might be wrong')
      
    elif date==None and location!=None:
      print('sorry your  date might be wrong, l can only give you weather in the next 7 days')
    else:
      print('sorry your location and date might be wrong, l can only give you weather in the next 7 days')



In [0]:
x=0
while x==0:
  test_text = input ("Chat with weather chatbot : ")
  print(agent([test_text]))
  a=str(agent([test_text])[0])
  if a=='Let me check':
    run(test_text)
 


Chat with weather chatbot : hie
['Hi, how can l help you']
Chat with weather chatbot : what is the weather
['For which location and date, l can only forecast for 1 week']
Chat with weather chatbot : Moscow 9June2019
['Let me check']
sorry your  date might be wrong, l can only give you weather in the next 7 days
Chat with weather chatbot : Moscow 9June 2019
['Let me check']
In Summary  Light rain next Thursday, with high temperatures peaking at 90°F on Wednesday.
---
06/06/19: Clear throughout the day. Temp range: 56.04 - 84.98
